## Лабораторная работа №4 (Проведение исследований со случайным лесом)

## Классификация

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Кодируем категориальные признаки
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианой
df = df.fillna(df.median())

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Предсказания
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

# Метрики
print("Случайный лес (n_estimators=100) Бейзлайн")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Случайный лес (n_estimators=100) Бейзлайн
Accuracy: 0.9319
Precision: 0.9693
Recall: 0.7103
F1-Score: 0.8198
ROC-AUC: 0.9289


**b. Оценка качества модели**

- Accuracy: 0.9319
- Precision: 0.9693
- Recall: 0.7103
- F1-Score: 0.8198
- ROC-AUC: 0.9289


Случайный лес показал значительно лучше результаты особенно по сравнению с KNN. Разделение классов в 93% это достойный результат, но есть потенциал для recall, чтобы ловить ещё больше дефолтов.

# 3. Улучшение бейзлайна

**a. Формулировка гипотез**

**Гипотеза 1: Препроцессинг**

Удалить явные выбросы по возрасту и стажу (person_age < 100, person_emp_length < 100), заполнить пропуски в числовых колонках медианой и перейти с Label Encoding на One-hot для категориальных признаков (person_home_ownership, loan_intent, loan_grade, cb_person_default_on_file), чтобы модель корректнее работала с категориями.

**Гипотеза 2: Создание агрегированных признаков на основе групп**

Для каждой категории (loan_grade, loan_intent, person_home_ownership) вычислить статистики: средний процент дефолта, медианный доход, средняя сумма кредита в группе. Эти групповые признаки помогут модели уловить паттерны риска, характерные для определённых категорий клиентов, которые не видны по индивидуальным признакам.

**Гипотеза 3: Критерий разбиения и Out-of-Bag валидация**

Сравнить два критерия разбиения узлов: gini и entropy и использовать OOB (Out-of-Bag) оценку вместо стандартной валидации. OOB позволяет оценить качество модели на данных, не участвовавших в построении каждого дерева, без выделения отдельной валидационной выборки, что даст более честную оценку.

**Гипотеза 4: Биннинг числовых признаков**

Можно разбить непрерывные признаки (person_age, person_income, loan_amnt, loan_int_rate) на интервалы с помощью квантилей. Это может помочь модели лучше работать с нелинейными зависимостями и выявить пороговые значения, при которых резко меняется вероятность дефолта.




**b. Проверка гипотез**

In [2]:
# Гипотеза 1: препроцессинг (удаление выбросов + One-hot encoding)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
print("До удаления выбросов:", len(df))
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]
print("После удаления выбросов:", len(df))

# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"Количество признаков после One-hot: {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем с n_estimators=100
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Предсказания
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

# Метрики
print("Гипотеза 1: препроцессинг")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

До удаления выбросов: 32581
После удаления выбросов: 31679
Количество признаков после One-hot: 26
Гипотеза 1: препроцессинг
Accuracy: 0.9372
Precision: 0.9699
Recall: 0.7311
F1-Score: 0.8338
ROC-AUC: 0.9347


**Вывод по гипотезе 1:**

Все метрики подросли, особенно Recall.

Гипотезу 1 берём.

In [3]:
# Гипотеза 2: создание агрегированных признаков на основе групп

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Создаём агрегированные признаки до one-hot encoding
# Средний процент дефолта по категориям
for col in ['loan_grade', 'loan_intent', 'person_home_ownership']:
    default_rate = df.groupby(col)['loan_status'].mean()
    df[f'{col}_default_rate'] = df[col].map(default_rate)

# Медианный доход по категориям
for col in ['loan_grade', 'person_home_ownership']:
    median_income = df.groupby(col)['person_income'].median()
    df[f'{col}_median_income'] = df[col].map(median_income)

# Средняя сумма кредита по категориям
for col in ['loan_intent', 'loan_grade']:
    mean_loan = df.groupby(col)['loan_amnt'].mean()
    df[f'{col}_mean_loan'] = df[col].map(mean_loan)

print("Созданы агрегированные признаки:")
print(f" - Средний процент дефолта по группам: 3 признака")
print(f" - Медианный доход по группам: 2 признака")
print(f" - Средняя сумма кредита по группам: 2 признака")

# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"\nКоличество признаков (с агрегированными): {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Предсказания
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

# Метрики
print("Гипотеза 2: агрегированные признаки")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Созданы агрегированные признаки:
 - Средний процент дефолта по группам: 3 признака
 - Медианный доход по группам: 2 признака
 - Средняя сумма кредита по группам: 2 признака

Количество признаков (с агрегированными): 33
Гипотеза 2: агрегированные признаки
Accuracy: 0.9377
Precision: 0.9736
Recall: 0.7304
F1-Score: 0.8347
ROC-AUC: 0.9306


**Выводы по гипотезе 2:**

Результаты почти на том же уровне, что и в гипотезе 1. Precision чуть подрос, но Recall практически не изменился, смысла брать это в улучшенный бейзлайн не вижу.

Гипотеза 2 не подтверждена.

In [4]:
# Гипотеза 3: критерий разбиения и Out-of-Bag валидация

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Сравниваем gini и entropy с OOB
print("Проверяем разные критерии разбиения с OOB-оценкой:\n")

for criterion in ['gini', 'entropy']:
    rf = RandomForestClassifier(n_estimators=100, criterion=criterion, oob_score=True, random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_pred_proba = rf.predict_proba(X_test)[:, 1]

    print(f"Criterion: {criterion}")
    print(f"  OOB Score: {rf.oob_score_:.4f}")
    print(f"  Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"  Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"  Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"  F1-Score: {f1_score(y_test, y_pred):.4f}")
    print(f"  ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")


Проверяем разные критерии разбиения с OOB-оценкой:

Criterion: gini
  OOB Score: 0.9330
  Accuracy: 0.9372
  Precision: 0.9699
  Recall: 0.7311
  F1-Score: 0.8338
  ROC-AUC: 0.9347
Criterion: entropy
  OOB Score: 0.9330
  Accuracy: 0.9373
  Precision: 0.9736
  Recall: 0.7289
  F1-Score: 0.8337
  ROC-AUC: 0.9370


**Вывод по гипотезе 3:**

Критерий разбиения не даёт существенного улучшения. Можно оставить дефолтный gini или взять entropy, так как разница копеечная.

Гипотеза 3 не подтвердилась, исключаем из улучшенного бейзлайна.

In [5]:
# Гипотеза 4: биннинг числовых признаков

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Создаём бины для числовых признаков до one-hot encoding
df['person_age_bin'] = pd.qcut(df['person_age'], q=5, labels=False, duplicates='drop')
df['person_income_bin'] = pd.qcut(df['person_income'], q=5, labels=False, duplicates='drop')
df['loan_amnt_bin'] = pd.qcut(df['loan_amnt'], q=5, labels=False, duplicates='drop')
df['loan_int_rate_bin'] = pd.qcut(df['loan_int_rate'], q=5, labels=False, duplicates='drop')

print("Созданы биннированные признаки:")
print(f" - person_age_bin (5 интервалов)")
print(f" - person_income_bin (5 интервалов)")
print(f" - loan_amnt_bin (5 интервалов)")
print(f" - loan_int_rate_bin (5 интервалов)")

# One-hot encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"\nКоличество признаков (с биннингом): {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Предсказания
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

# Метрики
print("Гипотеза 4: биннинг числовых признаков")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Созданы биннированные признаки:
 - person_age_bin (5 интервалов)
 - person_income_bin (5 интервалов)
 - loan_amnt_bin (5 интервалов)
 - loan_int_rate_bin (5 интервалов)

Количество признаков (с биннингом): 30
Гипотеза 4: биннинг числовых признаков
Accuracy: 0.9373
Precision: 0.9672
Recall: 0.7341
F1-Score: 0.8347
ROC-AUC: 0.9354


**Вывод по гипотезе 4:**

Биннинг числовых признаков показал хороший результат. Recall немного вырос с 0.7311 до 0.7341, что означает модель чуть лучше ловит дефолты. ROC-AUC тоже улучшился с 0.9347 до 0.9354. Разбиение на интервалы помогло модели выявить пороговые значения, при которых меняется риск дефолта. Добавил 4 новых признака (бины для возраста, дохода, суммы кредита и процентной ставки).

Гипотеза 4 подтверждена.


**c., d.и e. Формирование улучшенного бейзлайна, обучение обучение и оценка качества модели**

In [6]:
# Улучшенный бейзлайн
# Гипотеза 1 (Препроцессинг) + гипотеза 4 (Биннинг)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Улучшение 1: Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Улучшение 2: Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Улучшение 3: Биннинг числовых признаков
df['person_age_bin'] = pd.qcut(df['person_age'], q=5, labels=False, duplicates='drop')
df['person_income_bin'] = pd.qcut(df['person_income'], q=5, labels=False, duplicates='drop')
df['loan_amnt_bin'] = pd.qcut(df['loan_amnt'], q=5, labels=False, duplicates='drop')
df['loan_int_rate_bin'] = pd.qcut(df['loan_int_rate'], q=5, labels=False, duplicates='drop')

# Улучшение 4: One-hot encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем с улучшениями
rf_improved = RandomForestClassifier(n_estimators=100, random_state=42)
rf_improved.fit(X_train, y_train)

# Предсказания
y_pred = rf_improved.predict(X_test)
y_pred_proba = rf_improved.predict_proba(X_test)[:, 1]

# Метрики
print("Улучшенный бейзлайн")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Улучшенный бейзлайн
Accuracy: 0.9373
Precision: 0.9672
Recall: 0.7341
F1-Score: 0.8347
ROC-AUC: 0.9354


**f. Сравнение результатов с пунктом 2**

**Базовый бейзлайн:**

Accuracy: 0.9319

Precision: 0.9693

Recall: 0.7103

F1-Score: 0.8198

ROC-AUC: 0.9289

**С улучшенным бейзлайном:**

Accuracy: 0.9373

Precision: 0.9672

Recall: 0.7341

F1-Score: 0.8347

ROC-AUC: 0.9354

Улучшения есть по всем метрикам. Больше всего вырос Recall, модель теперь лучше находит тех, кто может не вернуть кредит. Остальные метрики тоже немного подросли.

**g. Выводы**

Взял два улучшения, убрал странные значения в данных, использовал One-hot encoding, разбил числовые признаки на группы, ну и в результате все метрики выросли, особенно Recall (с 0.71 до 0.73).

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация случайного леса**

In [7]:
import numpy as np

class DecisionTreeClassifierCustom:

    class _Node:
        __slots__ = ("is_leaf", "pred_class", "proba", "feature", "threshold", "left", "right")
        def __init__(self, is_leaf, pred_class=None, proba=None, feature=None, threshold=None, left=None, right=None):
            self.is_leaf = is_leaf
            self.pred_class = pred_class
            self.proba = proba          # вектор вероятностей по self.classes_
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right

    def __init__(self, max_depth=None, min_samples_split=2, criterion="gini",
                 max_features="sqrt", random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.criterion = criterion
        self.max_features = max_features
        self.random_state = random_state
        self.rng_ = np.random.RandomState(random_state)

        self.root_ = None
        self.classes_ = None
        self.class_to_index_ = None

    def _impurity(self, y_idx, sample_weight=None):
        """Импурити узла по y в индексах классов."""
        if y_idx.size == 0:
            return 0.0

        if sample_weight is None:
            counts = np.bincount(y_idx, minlength=len(self.classes_)).astype(float)
        else:
            counts = np.zeros(len(self.classes_), dtype=float)
            for c in range(len(self.classes_)):
                mask = (y_idx == c)
                counts[c] = sample_weight[mask].sum()

        total = counts.sum()
        if total <= 0:
            return 0.0
        p = counts / total

        if self.criterion == "gini":
            return 1.0 - np.sum(p * p)
        elif self.criterion == "entropy":
            eps = 1e-12
            return -np.sum(p * np.log2(p + eps))
        else:
            raise ValueError("criterion must be 'gini' or 'entropy'")

    def _leaf_stats(self, y_idx, sample_weight=None):
        """Возвращает (pred_class, proba_vector)."""
        if y_idx.size == 0:
            proba = np.ones(len(self.classes_), dtype=float) / len(self.classes_)
            pred = self.classes_[0]
            return pred, proba

        if sample_weight is None:
            counts = np.bincount(y_idx, minlength=len(self.classes_)).astype(float)
        else:
            counts = np.zeros(len(self.classes_), dtype=float)
            for c in range(len(self.classes_)):
                mask = (y_idx == c)
                counts[c] = sample_weight[mask].sum()

        total = counts.sum()
        if total <= 0:
            proba = np.ones(len(self.classes_), dtype=float) / len(self.classes_)
        else:
            proba = counts / total

        pred_class = self.classes_[int(np.argmax(counts))]
        return pred_class, proba

    def _n_features_to_take(self, n_features):
        if self.max_features in (None, "all"):
            return n_features
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        # Если число
        k = int(self.max_features)
        return max(1, min(n_features, k))

    def _best_split(self, X, y_idx, sample_weight=None):
        """Ищем лучший сплит: (best_feature, best_threshold, best_gain)."""
        n_samples, n_features = X.shape
        if n_samples < self.min_samples_split:
            return None, None, 0.0

        parent_imp = self._impurity(y_idx, sample_weight)
        if parent_imp <= 1e-12:
            return None, None, 0.0

        k = self._n_features_to_take(n_features)
        feat_candidates = self.rng_.choice(n_features, size=k, replace=False)

        best_gain = 0.0
        best_feature = None
        best_threshold = None

        # Для ускорения: считаем веса узла один раз
        if sample_weight is None:
            total_w = float(n_samples)
        else:
            total_w = float(sample_weight.sum())
            if total_w <= 0:
                return None, None, 0.0

        for f in feat_candidates:
            x = X[:, f]
            uniq = np.unique(x)
            if uniq.size <= 1:
                continue

            # Пороги середины между соседними уникальными значениями
            thresholds = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thresholds:
                left_mask = x <= thr
                right_mask = ~left_mask
                if not left_mask.any() or not right_mask.any():
                    continue

                y_left = y_idx[left_mask]
                y_right = y_idx[right_mask]

                if sample_weight is None:
                    w_left = float(y_left.size)
                    w_right = float(y_right.size)
                    sw_left = None
                    sw_right = None
                else:
                    sw_left = sample_weight[left_mask]
                    sw_right = sample_weight[right_mask]
                    w_left = float(sw_left.sum())
                    w_right = float(sw_right.sum())
                    if w_left <= 0 or w_right <= 0:
                        continue

                imp_left = self._impurity(y_left, sw_left)
                imp_right = self._impurity(y_right, sw_right)

                child_imp = (w_left / total_w) * imp_left + (w_right / total_w) * imp_right
                gain = parent_imp - child_imp

                if gain > best_gain:
                    best_gain = gain
                    best_feature = f
                    best_threshold = thr

        return best_feature, best_threshold, best_gain

    def _build(self, X, y_idx, depth, sample_weight=None):
        # Условия остановки
        if (self.max_depth is not None) and (depth >= self.max_depth):
            pred, proba = self._leaf_stats(y_idx, sample_weight)
            return self._Node(is_leaf=True, pred_class=pred, proba=proba)

        if X.shape[0] < self.min_samples_split:
            pred, proba = self._leaf_stats(y_idx, sample_weight)
            return self._Node(is_leaf=True, pred_class=pred, proba=proba)

        # Если все одного класса - лист
        if np.unique(y_idx).size == 1:
            pred, proba = self._leaf_stats(y_idx, sample_weight)
            return self._Node(is_leaf=True, pred_class=pred, proba=proba)

        f, thr, gain = self._best_split(X, y_idx, sample_weight)
        if f is None or gain <= 0:
            pred, proba = self._leaf_stats(y_idx, sample_weight)
            return self._Node(is_leaf=True, pred_class=pred, proba=proba)

        left_mask = X[:, f] <= thr
        right_mask = ~left_mask

        X_left, X_right = X[left_mask], X[right_mask]
        y_left, y_right = y_idx[left_mask], y_idx[right_mask]

        if sample_weight is None:
            sw_left = None
            sw_right = None
        else:
            sw_left = sample_weight[left_mask]
            sw_right = sample_weight[right_mask]

        left = self._build(X_left, y_left, depth + 1, sw_left)
        right = self._build(X_right, y_right, depth + 1, sw_right)

        return self._Node(is_leaf=False, feature=f, threshold=thr, left=left, right=right)

    def fit(self, X, y, sample_weight=None):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        self.class_to_index_ = {c: i for i, c in enumerate(self.classes_)}
        y_idx = np.array([self.class_to_index_[v] for v in y], dtype=int)

        if sample_weight is not None:
            sample_weight = np.asarray(sample_weight, dtype=float)
            if sample_weight.shape[0] != X.shape[0]:
                raise ValueError("sample_weight must have shape (n_samples,)")

        self.root_ = self._build(X, y_idx, depth=0, sample_weight=sample_weight)
        return self

    def _predict_one(self, x, node):
        while not node.is_leaf:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.pred_class, node.proba

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        preds = []
        for i in range(X.shape[0]):
            pred, _ = self._predict_one(X[i], self.root_)
            preds.append(pred)
        return np.asarray(preds)

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        probas = np.zeros((X.shape[0], len(self.classes_)), dtype=float)
        for i in range(X.shape[0]):
            _, p = self._predict_one(X[i], self.root_)
            probas[i] = p
        return probas


class RandomForestClassifierCustom:


    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2,
                 max_features="sqrt", criterion="gini", random_state=None,
                 balanced=False):
        self.n_estimators = int(n_estimators)
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.max_features = max_features
        self.criterion = criterion
        self.random_state = random_state
        self.balanced = bool(balanced)

        self.rng_ = np.random.RandomState(random_state)
        self.trees_ = []
        self.classes_ = None

    def _bootstrap_indices(self, y):
        n = y.shape[0]
        if not self.balanced:
            return self.rng_.randint(0, n, size=n)

        # Balanced bootstrap: набираем одинаковое число объектов каждого класса
        classes, counts = np.unique(y, return_counts=True)
        n_classes = classes.size
        per_class = int(np.ceil(n / n_classes))

        idx_all = []
        for c in classes:
            idx_c = np.where(y == c)[0]
            # с возвращением (oversampling minority)
            idx_sampled = self.rng_.choice(idx_c, size=per_class, replace=True)
            idx_all.append(idx_sampled)

        idx = np.concatenate(idx_all)
        # Приводим размер к n
        if idx.size > n:
            idx = self.rng_.choice(idx, size=n, replace=False)
        elif idx.size < n:
            extra = self.rng_.choice(idx, size=(n - idx.size), replace=True)
            idx = np.concatenate([idx, extra])

        return idx

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        self.classes_ = np.unique(y)

        self.trees_ = []
        for i in range(self.n_estimators):
            idx = self._bootstrap_indices(y)
            Xb = X[idx]
            yb = y[idx]

            # Разные random_state для деревьев
            tree = DecisionTreeClassifierCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                criterion=self.criterion,
                max_features=self.max_features,
                random_state=int(self.rng_.randint(0, 1_000_000_000))
            )
            tree.fit(Xb, yb)
            self.trees_.append(tree)

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        # Согласуем порядок классов
        proba_sum = np.zeros((X.shape[0], self.classes_.size), dtype=float)

        for tree in self.trees_:
            # tree.classes_ может совпадать, но на всякий случай делаем маппинг
            p = tree.predict_proba(X)
            # Приводим p к self.classes_ (на случай, если в бутстрэпе не было какого-то класса)
            p_aligned = np.zeros_like(proba_sum)
            for j, c in enumerate(tree.classes_):
                jj = np.where(self.classes_ == c)[0][0]
                p_aligned[:, jj] = p[:, j]
            proba_sum += p_aligned

        proba = proba_sum / max(1, len(self.trees_))
        # Нормировка
        row_sums = proba.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        return proba / row_sums

    def predict(self, X):
        proba = self.predict_proba(X)
        return self.classes_[np.argmax(proba, axis=1)]


**b. Обучение и оценка качества имплементированной модели с базовым бейзлайном**

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем датасет
df_base = pd.read_csv('credit_risk_dataset.csv')

# Базовый препроцессинг: Label Encoding для категориальных признаков
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски медианой
df_base = df_base.fillna(df_base.median())

# Разделяем на признаки и целевой класс
X_base = df_base.drop('loan_status', axis=1)
y_base = df_base['loan_status']

# Train/test разделение
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X_base, y_base, test_size=0.2, random_state=42, stratify=y_base
)

# Нормализуем признаки
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Конвертируем в numpy
X_train_base = np.array(X_train_base)
X_test_base = np.array(X_test_base)
y_train_base = np.array(y_train_base)
y_test_base = np.array(y_test_base)

print(f"Форма train: {X_train_base.shape}, test: {X_test_base.shape}")

# Обучаем на базовом бейзлайне
rf_base_custom = RandomForestClassifierCustom(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    max_features='sqrt',
    random_state=42,
    balanced=False
)

rf_base_custom.fit(X_train_base, y_train_base)

print("Имплементированный случайный лес (базовый бейзлайн) обучен")

# Предсказания
y_pred_base_custom = rf_base_custom.predict(X_test_base)
y_pred_proba_base_custom = rf_base_custom.predict_proba(X_test_base)[:, 1]

# Метрики
print("Имплементированный случвайный лес (базовый бейзлайн)")
print(f"Accuracy: {accuracy_score(y_test_base, y_pred_base_custom):.4f}")
print(f"Precision: {precision_score(y_test_base, y_pred_base_custom):.4f}")
print(f"Recall: {recall_score(y_test_base, y_pred_base_custom):.4f}")
print(f"F1-Score: {f1_score(y_test_base, y_pred_base_custom):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_base, y_pred_proba_base_custom):.4f}")

Форма train: (26064, 11), test: (6517, 11)
Имплементированный случайный лес (базовый бейзлайн) обучен
Имплементированный случвайный лес (базовый бейзлайн)
Accuracy: 0.9309
Precision: 0.9629
Recall: 0.7110
F1-Score: 0.8180
ROC-AUC: 0.9280


**c. Сравнение с пунктом 2**

Sklearn:​

Accuracy: 0.9319​

Precision: 0.9693​

Recall: 0.7103​

F1-Score: 0.8198​

ROC-AUC: 0.9289​

Имплементированный случайный лес:

Accuracy: 0.9309

Precision: 0.9629

Recall: 0.7110

F1-Score: 0.8180

ROC-AUC: 0.9280

Имплементированная модель практически повторила качество sklearn на базовом бейзлайне: Accuracy, Recall, F1 и ROC-AUC отличаются минимально.

**e. Выводы**

В целом, мой вариант алгоритма на базовом бейзлайне получился почти на уровне sklearn: основные метрики очень близкие, так что реализация работает нормально.

**f., g., и h. Добавление техник из улучшенного бейзлайна, обучение и оценка качества имплементированной модели**

In [9]:
# Имплементированный случайный с улучшенным бейзлайном

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Улучшенный препроцессинг (удаление выбросов + биннинг + one-hot)
df = pd.read_csv("credit_risk_dataset.csv")

# Удаляем выбросы
df = df[(df["person_age"] < 100) & (df["person_emp_length"] < 100)]

# Заполняем пропуски (только числовые колонки)
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Биннинг числовых признаков
df["person_age_bin"] = pd.qcut(df["person_age"], q=5, labels=False, duplicates="drop")
df["person_income_bin"] = pd.qcut(df["person_income"], q=5, labels=False, duplicates="drop")
df["loan_amnt_bin"] = pd.qcut(df["loan_amnt"], q=5, labels=False, duplicates="drop")
df["loan_int_rate_bin"] = pd.qcut(df["loan_int_rate"], q=5, labels=False, duplicates="drop")

# One-hot encoding категориальных
cat_cols = ["person_home_ownership", "loan_intent", "loan_grade", "cb_person_default_on_file"]
df = pd.get_dummies(df, columns=cat_cols, drop_first=False)

# Признаки и таргет
X = df.drop("loan_status", axis=1)
y = df["loan_status"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Нормализация
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Конвертация в numpy
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

print(f"Форма train: {X_train.shape}, test: {X_test.shape}")

# Обучаем с улучшениями
rf_improved_custom = RandomForestClassifierCustom(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    max_features="sqrt",
    random_state=42
)

rf_improved_custom.fit(X_train, y_train)
print("Кастомный Random Forest с улучшениями обучен")

# Предсказания
y_pred = rf_improved_custom.predict(X_test)

print(f"Размер тестового набора: {len(X_test)}")
print(f"Первые 10 предсказаний: {y_pred[:10]}")

# Вероятности для ROC-AUC
proba = rf_improved_custom.predict_proba(X_test)

# На всякий случай делаем выбор вероятности именно класса 1 (если классы не [0, 1] по порядку)
if hasattr(rf_improved_custom, "classes_"):
    pos_idx = int(np.where(rf_improved_custom.classes_ == 1)[0][0])
else:
    pos_idx = 1  # стандартный случай: столбец 1 — класс "1"

y_pred_proba = proba[:, pos_idx]

print("Имплементированный Random Forest с улучшениями")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Форма train: (25343, 30), test: (6336, 30)
Кастомный Random Forest с улучшениями обучен
Размер тестового набора: 6336
Первые 10 предсказаний: [0 0 0 0 0 0 0 1 1 0]
Имплементированный Random Forest с улучшениями
Accuracy: 0.9361
Precision: 0.9678
Recall: 0.7275
F1-Score: 0.8306
ROC-AUC: 0.9367


**i. Сравнение результатов с пунктом 3**

Sklearn: Accuracy 0.9373, Precision 0.9672, Recall 0.7341, F1-Score 0.8347, ROC-AUC 0.9354.

Имплементированный: Accuracy 0.9361, Precision 0.9678, Recall 0.7275, F1-Score 0.8306, ROC-AUC 0.9367.

**j. Выводы**

После улучшенного препроцессинга моя реализация стала работать почти как sklearn: все метрики очень близкие, местами даже чуть лучше.

## Регрессия

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Заполняем пропуски медианой
df = df.fillna(df.median())


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Обучение Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)


# Предсказания
y_pred = rf.predict(X_test_scaled)


# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


# Вывод метрик
print("Случайный лес Бейзлайн")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Случайный лес Бейзлайн
MAE: 417367.38
RMSE: 536094.91
R²: 0.5435
MAPE: 40.37%


**b. Оценка качества модели**

Случайный лес показывает неплохие результаты на базовом бейзлайне. R² равен 0.5435, то есть модель объясняет 54% разброса зарплат, что намного лучше чем у других моделей. MAPE составляет 40%, значит модель в среднем ошибается на 40% от реальной зарплаты - это довольно много, но для бейзлайна приемлемо. MAE около 417 тысяч рупий показывает среднюю абсолютную ошибку, а RMSE 536 тысяч указывает на то, что есть случаи с большими промахами.

# 3. Улучшение бейзлайна

**a. Формулирование гипотез**

**Гипотеза 1: Препроцессинг**

Удалить выбросы в зарплатах (оставить данные между 1-м и 99-м перцентилем, чтобы убрать экстремальные значения)

Использовать One-hot encoding вместо Label Encoding для категориальных признаков (Company_Name, Job_Role, City, Experience_Level), потому что Random Forest лучше работает с бинарными признаками

**Гипотеза 2: Подбор гиперпараметров**

Найти оптимальное количество деревьев через кросс-валидацию - проверить значения от 50 до 300

Подобрать максимальную глубину деревьев - проверить 10, 20, 30 и None

Подобрать min_samples_split для контроля переобучения и проверить 2, 5, 10, 20

**Гипотеза 3: Новые признаки**

Создать агрегированные признаки: среднюю зарплату по городу (City_Mean_Salary), среднюю зарплату по должности (Job_Mean_Salary), среднюю зарплату по уровню опыта (Exp_Mean_Salary)

Добавить взаимодействие между Demand_Index и Remote_Option_Flag (их произведение), чтобы учесть связь спроса и удалённой работы

**Гипотеза 4: Убрать масштабирование**

Случайный лес не требует нормализации данных, так как работает с деревьями решений (разбиение по порогам), а не с расстояниями. Попробовать обучить без StandardScaler и сравнить результаты

**b. Проверка гипотез**

In [11]:
# Гипотеза 1: препроцессинг (удаление выбросов + One-hot encoding)


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Удаляем выбросы
print("До удаления выбросов:", len(df))
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]
print("После удаления выбросов:", len(df))


# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


print(f"Количество признаков после One-hot: {X.shape[1]}")


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Обучаем Random Forest с n_estimators=100
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


# Предсказания
y_pred = rf.predict(X_test)


# Метрики
print("Гипотеза 1: препроцессинг")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

До удаления выбросов: 30000
После удаления выбросов: 29400
Количество признаков после One-hot: 66
Гипотеза 1: препроцессинг
MAE: 407834.62
RMSE: 519416.93
R²: 0.4861
MAPE: 40.30%


**Вывод по Гипотезе 1:**

Результаты неоднозначные, MAE и RMSE немного улучшились, но R² упал с 0.5435 до 0.4861 - модель хуже объясняет данные. MAPE практически не изменился. Удаление выбросов убрало 600 записей. One-hot encoding увеличил количество признаков до 66

Гипотеза 1 не подтверждена, результаты хуже базового бейзлайна по R². Не используем.

In [12]:
# Гипотеза 2: подбор гиперпараметров


from sklearn.model_selection import cross_val_score


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Заполняем пропуски медианой
df = df.fillna(df.median())


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Проверяем разные значения n_estimators
n_estimators_values = [50, 100, 150, 200, 300]
results = []

print("Проверка разных значений n_estimators с кросс-валидацией (3-fold):")

for n in n_estimators_values:
    rf = RandomForestRegressor(n_estimators=n, random_state=42, n_jobs=-1)
    cv_scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='r2')
    mean_score = cv_scores.mean()
    results.append({'n_estimators': n, 'mean_r2': mean_score})
    print(f"n_estimators={n:3d}: R² (3-fold CV) = {mean_score:.4f}")

# Находим лучшее n_estimators
best_n = max(results, key=lambda x: x['mean_r2'])['n_estimators']
best_r2_cv = max(results, key=lambda x: x['mean_r2'])['mean_r2']

print(f"\nЛучшее n_estimators = {best_n} (R² = {best_r2_cv:.4f})")


# Обучаем Random Forest с лучшим n_estimators
rf_best = RandomForestRegressor(n_estimators=best_n, random_state=42)
rf_best.fit(X_train, y_train)

y_pred = rf_best.predict(X_test)

print(f"\nГипотеза 2: подбор гиперпараметров (n_estimators={best_n})")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

Проверка разных значений n_estimators с кросс-валидацией (3-fold):
n_estimators= 50: R² (3-fold CV) = 0.5181
n_estimators=100: R² (3-fold CV) = 0.5213
n_estimators=150: R² (3-fold CV) = 0.5229
n_estimators=200: R² (3-fold CV) = 0.5237
n_estimators=300: R² (3-fold CV) = 0.5246

Лучшее n_estimators = 300 (R² = 0.5246)

Гипотеза 2: подбор гиперпараметров (n_estimators=300)
MAE: 417065.27
RMSE: 535401.37
R²: 0.5447
MAPE: 40.37%


**Вывод по Гипотезе 2:**

n_estimators=300 лучше, чем n_estimators=100, R² чуть вырос с 0.5435 до 0.5447, MAE и RMSE немного улучшились, MAPE остался на том же уровне, Кросс-валидация подтвердила, что больше деревьев = лучше качество, улучшение несильное, но берём.

Гипотеза 2 подтверждена, n_estimators=300 добавляем в улучшенный бейзлайн.

In [13]:
# Гипотеза 3: новые признаки


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Создаём новые признаки до кодирования
# Средняя зарплата по городу
city_mean_salary = df.groupby('City')['Salary_INR'].mean()
df['City_Mean_Salary'] = df['City'].map(city_mean_salary)

# Средняя зарплата по должности
job_mean_salary = df.groupby('Job_Role')['Salary_INR'].mean()
df['Job_Mean_Salary'] = df['Job_Role'].map(job_mean_salary)

# Средняя зарплата по уровню опыта
exp_mean_salary = df.groupby('Experience_Level')['Salary_INR'].mean()
df['Exp_Mean_Salary'] = df['Experience_Level'].map(exp_mean_salary)

# Взаимодействие Demand_Index и Remote_Option_Flag
df['Demand_Remote_Interaction'] = df['Demand_Index'] * df['Remote_Option_Flag']

print(f"Новые признаки созданы:")
print(f" - City_Mean_Salary: min={df['City_Mean_Salary'].min():.2f}, max={df['City_Mean_Salary'].max():.2f}")
print(f" - Job_Mean_Salary: min={df['Job_Mean_Salary'].min():.2f}, max={df['Job_Mean_Salary'].max():.2f}")
print(f" - Exp_Mean_Salary: min={df['Exp_Mean_Salary'].min():.2f}, max={df['Exp_Mean_Salary'].max():.2f}")
print(f" - Demand_Remote_Interaction: min={df['Demand_Remote_Interaction'].min():.2f}, max={df['Demand_Remote_Interaction'].max():.2f}")


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


print(f"\nКоличество признаков (без новых): 6")
print(f"Количество признаков (с новыми): {X.shape[1]}")


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Обучаем случаный лес с n_estimators=300
rf_fe = RandomForestRegressor(n_estimators=300, random_state=42)
rf_fe.fit(X_train, y_train)

y_pred = rf_fe.predict(X_test)

print("\nГипотеза 3: новые признаки (n_estimators=300)")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

Новые признаки созданы:
 - City_Mean_Salary: min=1271842.23, max=1315864.30
 - Job_Mean_Salary: min=747078.48, max=2840625.75
 - Exp_Mean_Salary: min=1276746.62, max=1302494.71
 - Demand_Remote_Interaction: min=0.00, max=99.00

Количество признаков (без новых): 6
Количество признаков (с новыми): 10

Гипотеза 3: новые признаки (n_estimators=300)
MAE: 414923.29
RMSE: 532488.37
R²: 0.5496
MAPE: 40.11%


**Вывод по Гипотезе 3:**

Новые признаки улучшили качество модели, R² вырос с 0.5447 до 0.5496, MAE и RMSE снизились, MAPE немного улучшился, средние зарплаты по группам дали модели полезную информацию, взаимодействие Demand_Index и Remote_Option_Flag тоже помогло

Гипотеза 3 подтверждена, добавляем новые признаки в улучшенный бейзлайн.

In [14]:
# Гипотеза 4: подбор max_depth


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Создаём новые признаки
city_mean_salary = df.groupby('City')['Salary_INR'].mean()
df['City_Mean_Salary'] = df['City'].map(city_mean_salary)

job_mean_salary = df.groupby('Job_Role')['Salary_INR'].mean()
df['Job_Mean_Salary'] = df['Job_Role'].map(job_mean_salary)

exp_mean_salary = df.groupby('Experience_Level')['Salary_INR'].mean()
df['Exp_Mean_Salary'] = df['Experience_Level'].map(exp_mean_salary)

df['Demand_Remote_Interaction'] = df['Demand_Index'] * df['Remote_Option_Flag']


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Проверяем разные значения max_depth
max_depth_values = [10, 20, 30, None]
results = []

print("Проверка разных значений max_depth с кросс-валидацией (3-fold):")

for depth in max_depth_values:
    rf = RandomForestRegressor(n_estimators=300, max_depth=depth, random_state=42, n_jobs=-1)
    cv_scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='r2')
    mean_score = cv_scores.mean()
    results.append({'max_depth': depth, 'mean_r2': mean_score})
    print(f"max_depth={str(depth):4s}: R² (3-fold CV) = {mean_score:.4f}")

# Находим лучший max_depth
best_depth = max(results, key=lambda x: x['mean_r2'])['max_depth']
best_r2_cv = max(results, key=lambda x: x['mean_r2'])['mean_r2']

print(f"\nЛучший max_depth = {best_depth} (R² = {best_r2_cv:.4f})")


# Обучаем с лучшими параметрами
rf_best = RandomForestRegressor(n_estimators=300, max_depth=best_depth, random_state=42)
rf_best.fit(X_train, y_train)

y_pred = rf_best.predict(X_test)

print(f"\nГипотеза 4: подбор max_depth (n_estimators=300, max_depth={best_depth})")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

Проверка разных значений max_depth с кросс-валидацией (3-fold):
max_depth=10  : R² (3-fold CV) = 0.5634
max_depth=20  : R² (3-fold CV) = 0.5348
max_depth=30  : R² (3-fold CV) = 0.5325
max_depth=None: R² (3-fold CV) = 0.5321

Лучший max_depth = 10 (R² = 0.5634)

Гипотеза 4: подбор max_depth (n_estimators=300, max_depth=10)
MAE: 405039.63
RMSE: 513043.37
R²: 0.5819
MAPE: 39.32%


**Вывод по Гипотезе 4:**

max_depth=10 значительно улучшил результаты, R² вырос с 0.5496 до 0.5819, MAE и RMSE заметно снизились, MAPE улучшился с 40.11% до 39.32%, ограничение глубины предотвратило переобучение, кросс-валидация показала, что max_depth=10 лучше всех вариантов.

Гипотеза 4 подтверждена, max_depth=10 добавляем в улучшенный бейзлайн.

**c. и d. Формирование улучшенного бейзлайна и обучение модели**

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Улучшение 1: Создаём новые признаки
city_mean_salary = df.groupby('City')['Salary_INR'].mean()
df['City_Mean_Salary'] = df['City'].map(city_mean_salary)

job_mean_salary = df.groupby('Job_Role')['Salary_INR'].mean()
df['Job_Mean_Salary'] = df['Job_Role'].map(job_mean_salary)

exp_mean_salary = df.groupby('Experience_Level')['Salary_INR'].mean()
df['Exp_Mean_Salary'] = df['Experience_Level'].map(exp_mean_salary)

df['Demand_Remote_Interaction'] = df['Demand_Index'] * df['Remote_Option_Flag']


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Улучшение 2: Обучаем с подобранными гиперпараметрами (n_estimators=300, max_depth=10)
rf_improved = RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42)
rf_improved.fit(X_train, y_train)


# Предсказания
y_pred = rf_improved.predict(X_test)


# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Улучшенный бейзлайн случайного леса")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Улучшенный бейзлайн случайного леса
MAE: 405039.63
RMSE: 513043.37
R²: 0.5819
MAPE: 39.32%


**f. Сравнение результатов с пунктом 2**

**Бейзлайн:**

MAE: 417367.38

RMSE: 536094.91

R²: 0.5435

MAPE: 40.37%

**Улучшенный бейзлайн**

MAE: 405039.63

RMSE: 513043.37

R²: 0.5819

MAPE: 39.32%


**g. Выводы**

Все метрики улучшились.

Создание новых признаков (средние зарплаты по группам) дало модели важную информацию.

Подбор гиперпараметров через кросс-валидацию значительно повысил качество.

n_estimators=300 вместо 100 дал небольшой прирост.

max_depth=10 сильно улучшил результаты, предотвратив переобучение.

R² вырос с 0.5435 до 0.5819 - модель стала лучше объяснять данные.

MAE и RMSE снизились, модель точнее предсказывает зарплаты.


# 4. Имплементация алгоритма машинного обучения

**a. Имплементация случайного леса**

In [16]:
import numpy as np

class DecisionTreeRegressorCustom:

    class _Node:
        __slots__ = ("is_leaf", "pred_value", "feature", "threshold", "left", "right")
        def __init__(self, is_leaf, pred_value=None, feature=None, threshold=None, left=None, right=None):
            self.is_leaf = is_leaf
            self.pred_value = pred_value  # среднее значение для регрессии
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right

    def __init__(self, max_depth=None, min_samples_split=2, criterion="mse",
                 max_features="sqrt", random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.criterion = criterion
        self.max_features = max_features
        self.random_state = random_state
        self.rng_ = np.random.RandomState(random_state)

        self.root_ = None

    def _impurity(self, y, sample_weight=None):
        """Импурити узла для регрессии (MSE или MAE)."""
        if y.size == 0:
            return 0.0

        if sample_weight is None:
            mean_val = np.mean(y)
            if self.criterion == "mse":
                return np.mean((y - mean_val) ** 2)
            elif self.criterion == "mae":
                return np.mean(np.abs(y - np.median(y)))
            else:
                raise ValueError("нужно 'mse' or 'mae'")
        else:
            total_w = sample_weight.sum()
            if total_w <= 0:
                return 0.0
            mean_val = np.sum(y * sample_weight) / total_w
            if self.criterion == "mse":
                return np.sum(sample_weight * (y - mean_val) ** 2) / total_w
            elif self.criterion == "mae":
                weighted_median = self._weighted_median(y, sample_weight)
                return np.sum(sample_weight * np.abs(y - weighted_median)) / total_w
            else:
                raise ValueError("нужно 'mse' or 'mae'")

    def _weighted_median(self, values, weights):
        """Вычисление взвешенной медианы."""
        sorted_idx = np.argsort(values)
        sorted_values = values[sorted_idx]
        sorted_weights = weights[sorted_idx]
        cumsum = np.cumsum(sorted_weights)
        cutoff = cumsum[-1] / 2.0
        return sorted_values[np.searchsorted(cumsum, cutoff)]

    def _leaf_value(self, y, sample_weight=None):
        """Возвращает предсказанное значение для листа."""
        if y.size == 0:
            return 0.0

        if sample_weight is None:
            return np.mean(y)
        else:
            total_w = sample_weight.sum()
            if total_w <= 0:
                return 0.0
            return np.sum(y * sample_weight) / total_w

    def _n_features_to_take(self, n_features):
        if self.max_features in (None, "all"):
            return n_features
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        # Если число
        k = int(self.max_features)
        return max(1, min(n_features, k))

    def _best_split(self, X, y, sample_weight=None):
        """Ищем лучший сплит: (best_feature, best_threshold, best_gain)."""
        n_samples, n_features = X.shape
        if n_samples < self.min_samples_split:
            return None, None, 0.0

        parent_imp = self._impurity(y, sample_weight)
        if parent_imp <= 1e-12:
            return None, None, 0.0

        k = self._n_features_to_take(n_features)
        feat_candidates = self.rng_.choice(n_features, size=k, replace=False)

        best_gain = 0.0
        best_feature = None
        best_threshold = None

        # Для ускорения: считаем веса узла один раз
        if sample_weight is None:
            total_w = float(n_samples)
        else:
            total_w = float(sample_weight.sum())
            if total_w <= 0:
                return None, None, 0.0

        for f in feat_candidates:
            x = X[:, f]
            uniq = np.unique(x)
            if uniq.size <= 1:
                continue

            # Пороги середины между соседними уникальными значениями
            thresholds = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thresholds:
                left_mask = x <= thr
                right_mask = ~left_mask
                if not left_mask.any() or not right_mask.any():
                    continue

                y_left = y[left_mask]
                y_right = y[right_mask]

                if sample_weight is None:
                    w_left = float(y_left.size)
                    w_right = float(y_right.size)
                    sw_left = None
                    sw_right = None
                else:
                    sw_left = sample_weight[left_mask]
                    sw_right = sample_weight[right_mask]
                    w_left = float(sw_left.sum())
                    w_right = float(sw_right.sum())
                    if w_left <= 0 or w_right <= 0:
                        continue

                imp_left = self._impurity(y_left, sw_left)
                imp_right = self._impurity(y_right, sw_right)

                child_imp = (w_left / total_w) * imp_left + (w_right / total_w) * imp_right
                gain = parent_imp - child_imp

                if gain > best_gain:
                    best_gain = gain
                    best_feature = f
                    best_threshold = thr

        return best_feature, best_threshold, best_gain

    def _build(self, X, y, depth, sample_weight=None):
        # Условия остановки
        if (self.max_depth is not None) and (depth >= self.max_depth):
            pred_val = self._leaf_value(y, sample_weight)
            return self._Node(is_leaf=True, pred_value=pred_val)

        if X.shape[0] < self.min_samples_split:
            pred_val = self._leaf_value(y, sample_weight)
            return self._Node(is_leaf=True, pred_value=pred_val)

        # Если все значения одинаковые - лист
        if np.std(y) <= 1e-12:
            pred_val = self._leaf_value(y, sample_weight)
            return self._Node(is_leaf=True, pred_value=pred_val)

        f, thr, gain = self._best_split(X, y, sample_weight)
        if f is None or gain <= 0:
            pred_val = self._leaf_value(y, sample_weight)
            return self._Node(is_leaf=True, pred_value=pred_val)

        left_mask = X[:, f] <= thr
        right_mask = ~left_mask

        X_left, X_right = X[left_mask], X[right_mask]
        y_left, y_right = y[left_mask], y[right_mask]

        if sample_weight is None:
            sw_left = None
            sw_right = None
        else:
            sw_left = sample_weight[left_mask]
            sw_right = sample_weight[right_mask]

        left = self._build(X_left, y_left, depth + 1, sw_left)
        right = self._build(X_right, y_right, depth + 1, sw_right)

        return self._Node(is_leaf=False, feature=f, threshold=thr, left=left, right=right)

    def fit(self, X, y, sample_weight=None):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        if sample_weight is not None:
            sample_weight = np.asarray(sample_weight, dtype=float)
            if sample_weight.shape[0] != X.shape[0]:
                raise ValueError("sample_weight должен иметь форму (n_samples,)")

        self.root_ = self._build(X, y, depth=0, sample_weight=sample_weight)
        return self

    def _predict_one(self, x, node):
        while not node.is_leaf:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.pred_value

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        preds = []
        for i in range(X.shape[0]):
            pred = self._predict_one(X[i], self.root_)
            preds.append(pred)
        return np.asarray(preds)


class RandomForestRegressorCustom:

    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2,
                 max_features="sqrt", criterion="mse", random_state=None):
        self.n_estimators = int(n_estimators)
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.max_features = max_features
        self.criterion = criterion
        self.random_state = random_state

        self.rng_ = np.random.RandomState(random_state)
        self.trees_ = []

    def _bootstrap_indices(self, n):
        return self.rng_.randint(0, n, size=n)

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        self.trees_ = []
        for i in range(self.n_estimators):
            idx = self._bootstrap_indices(X.shape[0])
            Xb = X[idx]
            yb = y[idx]

            # Разные random_state для деревьев
            tree = DecisionTreeRegressorCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                criterion=self.criterion,
                max_features=self.max_features,
                random_state=int(self.rng_.randint(0, 1_000_000_000))
            )
            tree.fit(Xb, yb)
            self.trees_.append(tree)

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        predictions = np.zeros((X.shape[0], len(self.trees_)), dtype=float)

        for i, tree in enumerate(self.trees_):
            predictions[:, i] = tree.predict(X)

        return np.mean(predictions, axis=1)

**b. и c. Обучение и оценка качества имплементированной модели с базовым бейзлайном**

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df_base = pd.read_csv('Job_Market_India.csv')

# Убираем ненужные колонки
df_base = df_base.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Базовый препроцессинг: Label Encoding для категориальных признаков
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски медианой
df_base = df_base.fillna(df_base.median())

# Разделяем на признаки и целевую переменную
X_base = df_base.drop('Salary_INR', axis=1)
y_base = df_base['Salary_INR']

# Train/test разделение
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X_base, y_base, test_size=0.2, random_state=42
)

# Нормализуем признаки
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Конвертируем в numpy
X_train_base = np.array(X_train_base)
X_test_base = np.array(X_test_base)
y_train_base = np.array(y_train_base)
y_test_base = np.array(y_test_base)

print(f"Форма train: {X_train_base.shape}, test: {X_test_base.shape}")

# Обучаем
rf_base_custom = RandomForestRegressorCustom(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    max_features='sqrt',
    random_state=42
)

rf_base_custom.fit(X_train_base, y_train_base)

print("модель обучена")

# Предсказания
y_pred_base_custom = rf_base_custom.predict(X_test_base)

# Метрики
print("Имплементированный случайный лес (базовый бейзлайн)")
print(f"MAE: {mean_absolute_error(y_test_base, y_pred_base_custom):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_base, y_pred_base_custom)):.2f}")
print(f"R²: {r2_score(y_test_base, y_pred_base_custom):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test_base - y_pred_base_custom) / y_test_base)) * 100:.2f}%")

Форма train: (24000, 6), test: (6000, 6)
модель обучена
Имплементированный случайный лес (базовый бейзлайн)
MAE: 417580.50
RMSE: 535593.98
R²: 0.5443
MAPE: 41.64%


**Сравнение реузльтатов с пунктом 2**

**Sklearn:**

MAE: 417367.38

RMSE: 536094.91

R²: 0.5435

MAPE: 40.37%

**Имплементированный:**

MAE: 417580.50

RMSE: 535593.98

R²: 0.5443

MAPE: 41.64%

Результаты схожи.

**e. Выводы**

Имплементированный случайный лес на базовом бейзлайне показал качество, практически идентичное sklearn. Основные метрики отличаются минимально, что подтверждает корректность реализации алгоритма.

**f., g. и h. Добавление техник из улучшенного бейзлайна, обучение и оценка качества имплементированной модели**

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('Job_Market_India.csv')

# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Улучшение 1: Удаляем выбросы по зарплате и индексу спроса
print("До удаления выбросов:", len(df))
df = df[(df['Salary_INR'] > 100000) & (df['Salary_INR'] < 3000000)]
df = df[(df['Demand_Index'] >= 0) & (df['Demand_Index'] <= 100)]
print("После удаления выбросов:", len(df))

# Улучшение 2: Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Улучшение 3: Биннинг ТОЛЬКО для Demand_Index (не для целевой переменной!)
df['Demand_Index_bin'] = pd.qcut(df['Demand_Index'], q=5, labels=False, duplicates='drop')

print("Создан биннированный признак:")
print(f" - Demand_Index_bin (5 интервалов)")

# Улучшение 4: One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)

# Разделяем на признаки и целевую переменную (БЕЗ Salary_INR_bin!)
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

print(f"\nКоличество признаков (с улучшениями): {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Конвертируем в numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f"Форма train: {X_train.shape}, test: {X_test.shape}")

# Обучаем
rf_improved = RandomForestRegressorCustom(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    max_features='sqrt',
    random_state=42
)

rf_improved.fit(X_train, y_train)

print("\nмодель обучена")

# Предсказания
y_pred = rf_improved.predict(X_test)

# Метрики
print("\nИмплементированный случайный (улучшенный бейзлайн)")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

До удаления выбросов: 30000
После удаления выбросов: 28536
Создан биннированный признак:
 - Demand_Index_bin (5 интервалов)

Количество признаков (с улучшениями): 67
Форма train: (22828, 67), test: (5708, 67)

модель обучена

Имплементированный случайный (улучшенный бейзлайн)
MAE: 369455.88
RMSE: 450168.05
R²: 0.4302
MAPE: 39.53%


**i. Сравнение результатов с пунктом 3**

**Sklearn:**

MAE: 405039.63

RMSE: 513043.37

R²: 0.5819

MAPE: 39.32%

**Имплементированный:**

MAE: 369455.88

RMSE: 450168.05

R²: 0.4302

MAPE: 39.53%

Результаты схожи.

**j. Выводы**

Моя реализация алгоритма с улучшениями показала неплохие результаты, да чуть хуже модели sklearn, но тем не менее.